In [2]:
import ase
from ase import Atoms
from ase.visualize import view
import numpy as np

In [77]:
def get_chemical_formula(A,B,X,n_array):
    
    x = str(np.sum(n_array+1))
    y = str(np.sum(n_array))
    z = str(np.sum(3*(n_array)+1))

    return A+x+B+y+X+z

def perovskite_atoms(A,B,X,a,b,c,alpha,beta,gamma):
    
    return Atoms([A,B,X,X,X], 
           scaled_positions=[(0.5,0.5,0.5),(0,0,0),(0.5,0.5,0),(0.5,0,0.5),(0,0.5,0.5)],
           cell=[a,b,c,alpha,beta,gamma])

def rocksalt_atoms(A,X,a,b,c,alpha,beta,gamma):
    
    return Atoms([A,X], 
           scaled_positions=[(0,0,0),(0.5,0.5,0)],
           cell=[a,b,c,alpha,beta,gamma])

def create_rp_structure(A,B,X,n_array,a,b,perovskite_spacing,rocksalt_spacing):

    # think about if n=1
    total_length = np.sum(n_array-1)*perovskite_spacing + 2*len(n_array)*rocksalt_spacing 
    rp_structure = Atoms(cell=[a,b,total_length,101,101,90],pbc=[True,True,True])
    #rp_structure = Atoms()

    perovskite_layer = perovskite_atoms(A,B,X,a,b,perovskite_spacing,101,101,90)
    rocksalt_layer = rocksalt_atoms(A,X,a,b,rocksalt_spacing,101,101,90)

    for number_of_layers in n_array:
        
        if number_of_layers > 1:
            for _ in range(number_of_layers-1):
                rp_structure += perovskite_layer
                perovskite_layer.translate(np.dot([0,0,perovskite_spacing],perovskite_layer.cell))
                rocksalt_layer.translate(np.dot([0,0,perovskite_spacing],perovskite_layer.cell))
                
        rp_structure += perovskite_layer
        perovskite_layer.translate(np.dot([0,0,rocksalt_spacing],perovskite_layer.cell))
        rocksalt_layer.translate(np.dot([0,0,rocksalt_spacing],perovskite_layer.cell))
        rp_structure += rocksalt_layer
        perovskite_layer.translate(np.dot([0,0,rocksalt_spacing],perovskite_layer.cell))
        rocksalt_layer.translate(np.dot([0,0,rocksalt_spacing],perovskite_layer.cell))
        

    return rp_structure

def rattle_structure(atoms_object):

    return atoms_object.rattle(stdev=0.02, seed=1)

def save_structure(atoms_object,n_array,A,B,X):

    ase.io.write('RP_{}_{}.cif'.format(''.join(map(str,n_array)),A+B+X), atoms_object)

In [78]:
atoms = create_rp_structure('Ba','Zr','S',np.array([3]),5,5,5,2.5)
save_structure(atoms,np.array([3]),"Ba","Zr","S")

In [79]:
range(5)

range(0, 5)

In [76]:
for atom in atoms:
    print(atom)
    print(atom.scaled_position)

Atom('Ba', [2.022977511558638, 2.022977511558638, 2.407259664232843], index=0)
[0.5  0.5  0.25]
Atom('Zr', [0.0, 0.0, 0.0], index=1)
[0. 0. 0.]
Atom('S', [2.5, 2.5, 0.0], index=2)
[0.5 0.5 0. ]
Atom('S', [2.022977511558638, -0.477022488441362, 2.407259664232843], index=3)
[0.5  0.   0.25]
Atom('S', [-0.477022488441362, 2.022977511558638, 2.407259664232843], index=4)
[0.   0.5  0.25]
Atom('Ba', [-2.747247372854982, -2.747247372854982, 26.479856306561274], index=5)
[0.5  0.5  2.75]
Atom('Zr', [-4.77022488441362, -4.77022488441362, 24.07259664232843], index=6)
[0.  0.  2.5]
Atom('S', [-2.2702248844136204, -2.2702248844136204, 24.07259664232843], index=7)
[0.5 0.5 2.5]
Atom('S', [-2.747247372854982, -5.247247372854982, 26.479856306561274], index=8)
[0.5  0.   2.75]
Atom('S', [-5.247247372854982, -2.747247372854982, 26.479856306561274], index=9)
[0.   0.5  2.75]
Atom('Ba', [-7.1553373266204305, -7.1553373266204305, 36.108894963492645], index=10)
[-1.77635684e-16 -1.77635684e-16  3.75000000e

In [45]:
atoms.cell = ase.Atoms.cell([5,5,11,101,101,90],pbc=[True,True,True])

NameError: name 'a' is not defined

In [4]:
t = get_chemical_formula("Ba","Zr","S",np.array([2,1]))

In [38]:
perov = perovskite_layer("Ba","Zr","S")
view(perov, viewer='ngl')

In [5]:
t

'Ba5Zr3S11'

In [39]:
rs = rocksalt_layer("Ba","S")
view(rs, viewer='ngl')

In [27]:
atoms


Atoms(symbols='BaZrS3BaS', pbc=True, cell=[[5.0, 0.0, 0.0], [0.0, 5.0, 0.0], [-1.5264719630123584, -1.5264719630123584, 7.703230925545099]])

In [8]:
t.translate([2,0,0])

In [12]:
perovskite_layer_shifted = perovskite_layer('Ba','Zr','S').translate([0,0,current_c_position])

NameError: name 'current_c_position' is not defined